In [61]:
import pandas as pd
import numpy as np
from utils import get_most_popular_diagnoses
from tqdm import tqdm
import pickle

In [2]:
train_df = pd.read_csv('./data/train_data_complaints_repeats_doctors.csv', index_col=0)

In [24]:
diagnoses = get_most_popular_diagnoses(train_df, 0.8)
diags_to_id = {diag:i for i, diag in enumerate(diagnoses)}

In [25]:
train_df['Id_диагноза'] = train_df.Код_диагноза.apply(
    lambda x: diags_to_id[x] if x in diags_to_id else len(diags_to_id))

In [47]:
gb = train_df.groupby('Id_Пациента')
onehot_dict = dict()
for key, df in tqdm(gb):
    onehot_dict[key] = np.bincount(df.Id_диагноза,minlength=len(diags_to_id)+1) / len(df)

100%|██████████| 29989/29989 [00:03<00:00, 7917.72it/s]


In [51]:
train_df['Id_диагноза_onehot'] = train_df.Id_Пациента.apply(lambda x:onehot_dict[x])

In [58]:
train_patient_arr = np.array(list(train_df.Id_диагноза_onehot))

In [62]:
with open('./data/diags_to_inds.pkl', 'wb') as f:
    pickle.dump(diags_to_id,f)

In [63]:
np.save('./data/patients_diagnoses.npy',train_patient_arr)